<a href="https://colab.research.google.com/github/stutii21/Cryptographic-Image-Transfer/blob/main/Cryptographic_Image_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Key Generation**




This part generates a pair of RSA keys: a public key (used for encryption) and a private key (used for decryption). It saves these keys to separate files.


In [ ]:
def generate_keys():
    key = RSA.generate(2048)  # Generate a 2048-bit RSA key pair
    private_key = key.export_key()  # Export the private key
    public_key = key.publickey().export_key()  # Export the public key

    with open("private_key.pem", "wb") as priv_file:
        priv_file.write(private_key)  # Save private key to a file

    with open("public_key.pem", "wb") as pub_file:
        pub_file.write(public_key)  # Save public key to a file


 **Image Encryption**


This function encrypts an image using the RSA public key.

In [ ]:
def encrypt_image(image_path, public_key_path):
    with open(public_key_path, "rb") as key_file:
        public_key = RSA.import_key(key_file.read())  # Load public key

    cipher_rsa = PKCS1_OAEP.new(public_key)  # Create an RSA cipher object

    with open(image_path, "rb") as image_file:
        image_data = image_file.read()  # Read image data

    encrypted_data = cipher_rsa.encrypt(image_data)  # Encrypt the image data

    encrypted_file_path = image_path + ".enc"
    with open(encrypted_file_path, "wb") as enc_file:
        enc_file.write(encrypted_data)  # Save encrypted data to a new file

    print(f"Image encrypted and saved to {encrypted_file_path}")
    return encrypted_file_path


**Image Decryption**


This function decrypts the encrypted image back into its original form using the RSA private key.



In [ ]:
def decrypt_image(encrypted_path, private_key_path):
    with open(private_key_path, "rb") as key_file:
        private_key = RSA.import_key(key_file.read())  # Load private key

    cipher_rsa = PKCS1_OAEP.new(private_key)  # Create an RSA cipher object

    with open(encrypted_path, "rb") as enc_file:
        encrypted_data = enc_file.read()  # Read encrypted data

    decrypted_data = cipher_rsa.decrypt(encrypted_data)  # Decrypt the data

    original_file_path = encrypted_path.replace(".enc", "_decrypted.png")
    with open(original_file_path, "wb") as dec_file:
        dec_file.write(decrypted_data)  # Save decrypted image

    print(f"Image decrypted and saved to {original_file_path}")
    return original_file_path


**Server for Sending the Encrypted Image**


The server reads the encrypted image and sends it to a connected client over a socket.



In [ ]:
def server(host="127.0.0.1", port=65432):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)
    print(f"Server listening on {host}:{port}")

    conn, addr = server_socket.accept()
    print(f"Connection established with {addr}")

    encrypted_path = "image_to_send.png.enc"
    with open(encrypted_path, "rb") as enc_file:
        conn.sendall(enc_file.read())  # Send encrypted image to client

    print("Encrypted image sent.")
    conn.close()
    server_socket.close()


**Client for Receiving the Encrypted Image**


The client receives the encrypted image sent by the server.

In [ ]:
def client(host="127.0.0.1", port=65432, save_path="received_image.png.enc"):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))  # Connect to server

    with open(save_path, "wb") as file:
        while True:
            data = client_socket.recv(4096)  # Receive data in chunks
            if not data:
                break
            file.write(data)  # Save received data to a file

    print(f"Encrypted image received and saved as {save_path}")
    client_socket.close()
    return save_path


 **Main Program**


The main script allows the user to choose between generating keys, encrypting, decrypting, or running the server/client.

In [ ]:
if __name__ == "__main__":
    print("1. Generate RSA Keys\n2. Encrypt Image\n3. Decrypt Image\n4. Run Server\n5. Run Client")
    choice = int(input("Enter your choice: "))

    if choice == 1:
        generate_keys()
    elif choice == 2:
        image_path = input("Enter image path: ")
        public_key_path = input("Enter public key path: ")
        encrypt_image(image_path, public_key_path)
    elif choice == 3:
        encrypted_path = input("Enter encrypted file path: ")
        private_key_path = input("Enter private key path: ")
        decrypt_image(encrypted_path, private_key_path)
    elif choice == 4:
        server()
    elif choice == 5:
        save_path = client()
        print(f"Saved encrypted file: {save_path}")
    else:
        print("Invalid choice.")


1. Generate RSA Keys
2. Encrypt Image
3. Decrypt Image
4. Run Server
5. Run Client
